## 簡介
本Colab筆記本展示如何使用LangGraph實作三個Agentic Pattern：ReAct（公司用語生成）、Reflection（電子郵件改寫）和Hierarchical Workflow（文章翻譯）。您將學會定義節點、邊和狀態，建構模組化AI工作流程，應用於正式用語生成、郵件優化和多語言翻譯。

**學習成果**：
- 掌握LangGraph的核心功能（節點、邊、狀態管理）。
- 實現公司用語、電子郵件改進和文章翻譯的AI代理。
- 理解AI從單一代理到協作流程的應用。

**先決條件**：熟悉Python，需提供OpenAI API金鑰以進行LLM調用。

執行以下程式碼，安裝必要套件並開始實作！

In [ ]:
!pip install -q langgraph openai

# 實作1：ReAct - 公司用語生成
本實作建構一個簡化的ReAct代理，生成正式的公司用語，例如將會議筆記轉為專業會議記錄。代理執行最多2次生成，確保穩定性，避免依賴LLM動態決策。

**目標：**
實現一個穩定的ReAct代理，生成公司用語。

**步驟：**
1. 定義狀態（輸入、歷史、結果、迭代次數）。
2. 創建生成節點（產生用語）、檢查節點（記錄迭代）。
3. 配置邊，固定2次迭代後結束。

In [ ]:
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from typing import Dict
from openai import OpenAI
from google.colab import userdata

@dataclass
class ReActState:
    input: str
    history: list
    result: str = ""
    iteration: int = 0

# 設定OpenAI API
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # 請替換為您的API金鑰

def llm_call(prompt: str, max_tokens: int = 100) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

def generate_node(state: ReActState) -> Dict:
    prompt = f"將以下內容轉為正式公司用語：\n{state.input}\n歷史：{state.history}"
    state.result = llm_call(prompt)
    state.history.append(f"生成：{state.result}")
    return {"result": state.result, "history": state.history}

def check_node(state: ReActState) -> Dict:
    state.iteration += 1
    return {"iteration": state.iteration}

workflow = StateGraph(ReActState)
workflow.add_node("generate", generate_node)
workflow.add_node("check", check_node)
workflow.set_entry_point("generate")
workflow.add_edge("generate", "check")
workflow.add_conditional_edges(
    "check",
    lambda state: "end" if state.iteration >= 2 else "generate",
    {"generate": "generate", "end": END}
)
graph = workflow.compile()

# 執行
result = graph.invoke(ReActState(input="會議討論了新產品發布", history=[]))
print(f"最終用語：{result['result']}")
print(f"生成歷史：{result['history']}")

最終用語：在本次會議中討論了公司新產品發布計畫。公司歷史紀錄中無相關資訊。
生成歷史：['生成：會議研討了新產品發布計畫。\n公司歷史：[]', '生成：在本次會議中討論了公司新產品發布計畫。公司歷史紀錄中無相關資訊。']


# 實作2：Reflection - 電子郵件改寫
本實作建構一個Reflection代理，改進電子郵件的語氣和結構，例如將草稿改為更專業的版本。代理透過LLM反思並優化郵件內容。

**目標**：實現一個Reflection代理，改進電子郵件。

**步驟：**
1. 定義狀態（草稿、郵件、反饋）。
2. 創建生成節點（改寫郵件）、反思節點（檢查問題）、更新節點（修正郵件）。
3. 配置邊與條件邏輯。

In [ ]:
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from typing import Dict
from openai import OpenAI

@dataclass
class ReflectionState:
    draft: str
    email: str = ""
    feedback: str = ""
    has_issue: bool = False

# 設定OpenAI API
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # 請替換為您的API金鑰

def llm_call(prompt: str, max_tokens: int = 200) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

def generate_node(state: ReflectionState) -> Dict:
    prompt = f"將以下電子郵件草稿改為專業語氣：\n{state.draft}"
    state.email = llm_call(prompt)
    return {"email": state.email}

def reflect_node(state: ReflectionState) -> Dict:
    prompt = f"檢查以下電子郵件是否存在語氣或結構問題：\n{state.email}\n若有問題，回覆具體建議；若無，回覆'無問題'"
    state.feedback = llm_call(prompt)
    state.has_issue = "無問題" not in state.feedback
    return {"feedback": state.feedback, "has_issue": state.has_issue}

def update_node(state: ReflectionState) -> Dict:
    prompt = f"基於以下建議修正電子郵件：\n建議：{state.feedback}\n原始郵件：\n{state.email}"
    state.email = llm_call(prompt)
    return {"email": state.email}

workflow = StateGraph(ReflectionState)
workflow.add_node("generate", generate_node)
workflow.add_node("reflect", reflect_node)
workflow.add_node("update", update_node)
workflow.set_entry_point("generate")
workflow.add_edge("generate", "reflect")
workflow.add_conditional_edges(
    "reflect",
    lambda state: "update" if state.has_issue else "end",
    {"update": "update", "end": END}
)
workflow.add_edge("update", "reflect")
graph = workflow.compile()

# 執行
result = graph.invoke(ReflectionState(draft="嗨，明天可以開會嗎？"))
print(f"最終郵件：\n{result['email']}")
print(f"反饋：{result['feedback']}")

最終郵件：
論非常有價值。

若您能參加，請回覆郵件，告訴我們您的方便時間，我們將安排會議時間。期待您的回覆。

謝謝！

XXX公司發展部門 敬上
反饋：這封電子郵件看起來沒有明顯的語氣或結構問題。建議可以加上更具體的內容，例如提供會議主題或目的，以便收件人更清楚了解。另外，要求收件人回覆具體建議或回覆'無問題'也是一個很好的做法。如果想要更加正式或專業，可以適當地修改郵件內容。


延伸練習：
- 將draft改為其他郵件草稿（例如「請審核文件」）。
- 檢查feedback的改進建議。
- 調整reflect_node的提示，檢查特定問題（例如清晰度）。



---



# 實作3：Hierarchical Workflow - 文章翻譯
本實作建構一個Hierarchical Workflow，模擬文章翻譯場景，例如將英文文章翻譯成中文並校對。監督代理分配翻譯或校對任務，子代理執行相應工作。

**目標**：實現一個層次化工作流程，完成文章翻譯與校對。

**步驟：**

1. 定義狀態（任務類型、原文、翻譯）。
2. 創建監督節點（分配任務）、翻譯節點（生成翻譯）、校對節點（檢查翻譯）。
3. 配置邊與邏輯。

In [ ]:
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from typing import Dict
from openai import OpenAI

@dataclass
class WorkflowState:
    task_type: str
    source: str = ""
    translation: str = ""
    agent: str = ""

# 設定OpenAI API
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # 請替換為您的API金鑰

def llm_call(prompt: str, max_tokens: int = 200) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

def supervisor_node(state: WorkflowState) -> Dict:
    prompt = f"任務類型：{state.task_type}\n選擇代理：僅回覆'translate_agent'或'proofread_agent'"
    response = llm_call(prompt, max_tokens=10)
    # 清理回應，確保僅為'translate_agent'或'proofread_agent'
    state.agent = 'translate_agent' if 'translate' in response.lower() else 'proofread_agent'
    return {"agent": state.agent}

def translate_node(state: WorkflowState) -> Dict:
    prompt = f"將以下英文翻譯成中文：\n{state.source}"
    state.translation = llm_call(prompt)
    state.task_type = "proofread"  # 模擬後續校對
    return {"translation": state.translation, "task_type": state.task_type}

def proofread_node(state: WorkflowState) -> Dict:
    prompt = f"校對以下翻譯，確保語法和語義正確：\n原文：{state.source}\n翻譯：{state.translation}"
    state.translation = llm_call(prompt)
    return {"translation": state.translation}

workflow = StateGraph(WorkflowState)
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("translate", translate_node)
workflow.add_node("proofread", proofread_node)
workflow.set_entry_point("supervisor")
workflow.add_conditional_edges(
    "supervisor",
    lambda state: state.agent,
    {"translate_agent": "translate", "proofread_agent": "proofread"}
)
workflow.add_edge("translate", "supervisor")
workflow.add_edge("proofread", END)
graph = workflow.compile()

# 執行
result = graph.invoke(WorkflowState(task_type="translate", source="AI will transform industries in 2025."))
print(f"原文：{result['source']}")
print(f"翻譯：{result['translation']}")

原文：AI will transform industries in 2025.
翻譯：校對後翻譯：2025年將會有人工智慧改變產業。


延伸練習
- 將source改為其他英文句子（例如「The future is bright」）。
- 檢查translation的內容。
- 添加新子代理（例如格式調整代理）。